In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import numpy as np
import pandas as pd
import gc

# ensemble:
* 1. 221
* 2. 224 
* 3. 238
* 4. 240
* 5. 219



In [ ]:
data_path = '/content/gdrive/MyDrive/567_final_proj/output'
sub0 = pd.read_csv(data_path + '/submission_1.csv').sort_values('customer_id').reset_index(drop=True)
sub1 = pd.read_csv(data_path + '/submission_2.csv').sort_values('customer_id').reset_index(drop=True)
sub2 = pd.read_csv(data_path + '/submission_3.csv').sort_values('customer_id').reset_index(drop=True)
sub3 = pd.read_csv(data_path + '/submission_4.csv').sort_values('customer_id').reset_index(drop=True)
sub4 = pd.read_csv(data_path + '/submission_5.csv').sort_values('customer_id').reset_index(drop=True)


In [ ]:
sub0.columns = ['customer_id', 'prediction0']
sub0['prediction1'] = sub1['prediction']
sub0['prediction2'] = sub2['prediction']
sub0['prediction3'] = sub3['prediction']
sub0['prediction4'] = sub4['prediction']

del sub1, sub2, sub3, sub4
gc.collect()
sub0.head()

,customer_id,prediction0,prediction1,prediction2,prediction3,prediction4
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0568601006 0745232001 0751471001 04...,0568601043 0751471001 0448509014 0918292001 09...,0568601043 0568601006 0751471001 0448509014 09...,0568601043 0568601006 0751471001 0924243001 04...,0568601043 0751471001 0448509014 0918292001 09...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0826211002 0739590027 0811835004 0764280001 07...,0751471001 0448509014 0918292001 0924243001 09...,0826211002 0924243001 0739590027 0800436010 09...,0826211002 0924243001 0924243002 0739590027 08...,0826211002 0751471001 0448509014 0918292001 09...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0852643001 0852643003 0727808007 08...,0794321007 0751471001 0448509014 0918292001 09...,0794321007 0852643001 0852643003 0858883002 09...,0794321007 0852643001 0924243001 0924243002 07...,0794321007 0751471001 0448509014 0918292001 09...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0751471001 0678942001 0673677002 0579541001 05...,0751471001 0448509014 0918292001 0924243001 09...,0448509014 0751471001 0924243001 0924243002 05...,0448509014 0924243002 0924243001 0751471001 05...,0751471001 0448509014 0918292001 0924243001 09...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0730683050 0791587015 0896152002 0927530004 05...,0751471001 0448509014 0918292001 0924243001 09...,0730683050 0791587015 0924243001 0896152002 09...,0730683050 0791587015 0927530004 0896152002 09...,0896152002 0730683050 0791587015 0818320001 09...


In [ ]:
# appending results

REC = []
customer_ids = []
for i in range(len(sub0['prediction0'])):
    res_sub = []
    res_sub.extend(sub0['prediction0'][i].split())
    res_sub.extend(sub0['prediction1'][i].split())
    res_sub.extend(sub0['prediction2'][i].split())
    res_sub.extend(sub0['prediction3'][i].split())
    res_sub.extend(sub0['prediction4'][i].split())
    res_sub = list(set(res_sub))
    REC.append(res_sub)
    customer_ids.append(sub0['customer_id'][i])
    

results_df = pd.DataFrame(list(zip(customer_ids, REC)), columns=['customer_id', 'prediction'])
results_df["prediction"] = results_df["prediction"].apply(lambda x: " ".join(x))
print(results_df.head())
results_df.to_csv('res_blend.csv', index=False)
    


In [ ]:
def cust_blend(dt, W = [1,1,1,1,1]):
    #Global ensemble weights
    #W = [1.15,0.95,0.85]

    #Create a list of all model predictions
    REC = []

    # Second Try
    REC.append(dt['prediction0'].split())
    REC.append(dt['prediction1'].split())
    REC.append(dt['prediction2'].split())
    REC.append(dt['prediction3'].split())
    REC.append(dt['prediction4'].split())
    # REC.append(dt['prediction5'].split())
    # REC.append(dt['prediction6'].split())
    # REC.append(dt['prediction7'].split())

    #Create a dictionary of items recommended.
    #Assign a weight according the order of appearance and multiply by global weights
    res = {}
    for M in range(len(REC)):
        for n, v in enumerate(REC[M]):
            if v in res:
                res[v] += (W[M]/(n+1))
            else:
                res[v] = (W[M]/(n+1))

    # Sort dictionary by item weights
    res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())

    # Return the top 12 items only
    return ' '.join(res[:12])


# 221, 224, 238, 240, 219
# .194, .196, .208, .210, .192  --> 239
# 0.75, 0.65, 0.65, 1.2, 0.6  --> 239
# 0.85, 1.05, 0.75, 1.05, 0.86 --> 238
# 1.2, 0.95, 0.75, 0.68, 0.78 --> 238
# 0.68, 0.80, 1.35, 1.0, 0.60 --> 238
# 0.75, 0.65, 1.0, 1.2, 0.6 --> 238
# 0.75, 0.75, 0.8, 1.3, 0.6 --> 239
sub0['prediction'] = sub0.apply(cust_blend, W= [0.75, 0.85, 1.0, 1.3, 0.65], axis=1)


,customer_id,prediction0,prediction1,prediction2,prediction3,prediction4,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0568601006 0745232001 0751471001 04...,0568601043 0751471001 0448509014 0918292001 09...,0568601043 0568601006 0751471001 0448509014 09...,0568601043 0568601006 0751471001 0924243001 04...,0568601043 0751471001 0448509014 0918292001 09...,0568601043 0751471001 0568601006 0448509014 09...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0826211002 0739590027 0811835004 0764280001 07...,0751471001 0448509014 0918292001 0924243001 09...,0826211002 0924243001 0739590027 0800436010 09...,0826211002 0924243001 0924243002 0739590027 08...,0826211002 0751471001 0448509014 0918292001 09...,0826211002 0924243001 0751471001 0739590027 04...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0852643001 0852643003 0727808007 08...,0794321007 0751471001 0448509014 0918292001 09...,0794321007 0852643001 0852643003 0858883002 09...,0794321007 0852643001 0924243001 0924243002 07...,0794321007 0751471001 0448509014 0918292001 09...,0794321007 0852643001 0751471001 0924243001 04...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0751471001 0678942001 0673677002 0579541001 05...,0751471001 0448509014 0918292001 0924243001 09...,0448509014 0751471001 0924243001 0924243002 05...,0448509014 0924243002 0924243001 0751471001 05...,0751471001 0448509014 0918292001 0924243001 09...,0751471001 0448509014 0924243001 0924243002 06...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0730683050 0791587015 0896152002 0927530004 05...,0751471001 0448509014 0918292001 0924243001 09...,0730683050 0791587015 0924243001 0896152002 09...,0730683050 0791587015 0927530004 0896152002 09...,0896152002 0730683050 0791587015 0818320001 09...,0730683050 0791587015 0896152002 0751471001 09...


In [ ]:
# del sub0['prediction0']
# del sub0['prediction1']
# del sub0['prediction2']
# del sub0['prediction3']
# del sub0['prediction4']
# gc.collect()
res = sub0[['customer_id', 'prediction']]
res.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0751471001 0568601006 0448509014 09...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0826211002 0924243001 0751471001 0739590027 04...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0852643001 0751471001 0924243001 04...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0751471001 0448509014 0924243001 0924243002 06...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0730683050 0791587015 0896152002 0751471001 09...


In [ ]:
res.to_csv('submission_random7.csv', index=False)